In [1]:
import pandas as pd
import numpy as np

path = '../data/ETHUSDT_5m.csv'
symbol = path.split('/')[-1].split('_')[0].replace('USDT', '')
init_cash = 2000
df = pd.read_csv(path)
df.head()

,time,open,high,low,close,volume
0,2023-09-20 00:00:00,1642.48,1642.64,1641.43,1641.43,3635.365
1,2023-09-20 00:05:00,1641.43,1641.76,1640.80,1641.76,4449.485
2,2023-09-20 00:10:00,1641.75,1643.70,1641.61,1642.99,4339.614
3,2023-09-20 00:15:00,1642.99,1643.15,1641.64,1641.65,2065.457
4,2023-09-20 00:20:00,1641.64,1641.84,1640.32,1640.45,2349.314


In [2]:
import pandas_ta as ta
from pycaret.anomaly import load_model, predict_model

#load model
model = load_model('anomaly_detector')

#predict
df['Predict'] = predict_model(model, data=df)['Anomaly']
df['Signal'] = df['Predict']
df['ATR'] = df.ta.atr()
df['ADX'] = df.ta.adx()['ADX_14']
#help(ta.adx)

Transformation Pipeline and Model Successfully Loaded


In [3]:
df.ta.cdl_pattern(name="longline").value_counts()

CDL_LONGLINE
 0.0            8262
 100.0          1043
-100.0           966
dtype: int64

In [4]:
help(ta.mad)

Help on function mad in module pandas_ta.statistics.mad:

mad(close, length=None, offset=None, **kwargs)
    Rolling Mean Absolute Deviation
    
    Sources:
    
    Calculation:
        Default Inputs:
            length=30
        mad = close.rolling(length).mad()
    
    Args:
        close (pd.Series): Series of 'close's
        length (int): It's period. Default: 30
        offset (int): How many periods to offset the result. Default: 0
    
    Kwargs:
        fillna (value, optional): pd.DataFrame.fillna(value)
        fill_method (value, optional): Type of fill method
    
    Returns:
        pd.Series: New feature generated.



In [5]:
#dropna
df = df.dropna()
df.head()

,time,open,high,low,close,volume,Predict,Signal,ATR,ADX
27,2023-09-20 02:15:00,1643.39,1644.11,1642.63,1644.04,3113.746,0,0,1.701942,31.774769
28,2023-09-20 02:20:00,1644.04,1644.04,1641.45,1641.45,3217.323,0,0,1.765375,29.850188
29,2023-09-20 02:25:00,1641.45,1642.25,1640.92,1641.76,5188.831,0,0,1.734277,28.613207
30,2023-09-20 02:30:00,1641.75,1642.28,1641.04,1641.71,1903.762,0,0,1.698971,27.503110
31,2023-09-20 02:35:00,1641.72,1641.72,1639.05,1639.95,7381.151,0,0,1.768330,27.907182


In [6]:
#set time to index

df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df.head()

,open,high,low,close,volume,Predict,Signal,ATR,ADX
time,,,,,,,,,
2023-09-20 02:15:00,1643.39,1644.11,1642.63,1644.04,3113.746,0,0,1.701942,31.774769
2023-09-20 02:20:00,1644.04,1644.04,1641.45,1641.45,3217.323,0,0,1.765375,29.850188
2023-09-20 02:25:00,1641.45,1642.25,1640.92,1641.76,5188.831,0,0,1.734277,28.613207
2023-09-20 02:30:00,1641.75,1642.28,1641.04,1641.71,1903.762,0,0,1.698971,27.503110
2023-09-20 02:35:00,1641.72,1641.72,1639.05,1639.95,7381.151,0,0,1.768330,27.907182


In [7]:
#rename columns
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

In [8]:
df['Signal'].value_counts()

0    9729
1     515
Name: Signal, dtype: int64

In [9]:
from backtesting import Strategy
from backtesting import Backtest

def SIGNAL(df,col_name):
    return df[col_name]

class Scalping_Strategy(Strategy):
    atr_sl_rate = 1.3
    TPSL_rate = 1.5
    save_size_rate = 0.125
    init_cash = init_cash
    profit_target = 500
    size_multiplier_rate = 0.5

    def init(self):
        super().init()
        self.signal = self.I(SIGNAL, self.data, 'Signal')

    def next(self):
        super().next()

        slatr = self.atr_sl_rate * self.data.ATR[-1]
        size_multiplier = self.size_multiplier_rate if self.equity >= self.profit_target else 1.0

        if self.signal == 1:
            sl_price = self.data.Close[-1] - slatr
            tp_price = self.data.Close[-1] + self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity > self.init_cash * 1.5:
                self.buy(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.buy(sl=sl_price, tp=tp_price)

        if self.signal == -1:
            sl_price = self.data.Close[-1] + slatr
            tp_price = self.data.Close[-1] - self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity >= self.init_cash * 1.5:
                self.sell(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.sell(sl=sl_price, tp=tp_price)
            

bt = Backtest(df, Scalping_Strategy, cash=init_cash, commission=.00002, margin=.05)
stat = bt.run()
stat

Start                     2023-09-20 02:15:00
End                       2023-10-25 15:50:00
Duration                     35 days 13:35:00
Exposure Time [%]                   21.885982
Equity Final [$]                  5534.888993
Equity Peak [$]                   8050.284246
Return [%]                          176.74445
Buy & Hold Return [%]               10.221771
Return (Ann.) [%]                959318.88577
Volatility (Ann.) [%]     8413611042244793...
Sharpe Ratio                              0.0
Sortino Ratio                     4341.613248
Calmar Ratio                     11401.539373
Max. Drawdown [%]                  -84.139418
Avg. Drawdown [%]                  -19.251949
Max. Drawdown Duration       24 days 07:40:00
Avg. Drawdown Duration        2 days 00:34:00
# Trades                                  362
Win Rate [%]                         45.58011
Best Trade [%]                       1.599523
Worst Trade [%]                      -1.04392
Avg. Trade [%]                    

In [10]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stat['_equity_curve'].index, y=stat['_equity_curve']['Equity'])])
fig.show()

In [11]:
%%time
stats = bt.optimize(atr_sl_rate = np.arange(0.3, 2.6, 0.2).tolist(),
                    TPSL_rate = np.arange(0.3, 2.6, 0.2).tolist(),
                    size_multiplier_rate = np.arange(0.1, 0.5, 0.1).tolist(),
                    maximize='Equity Final [$]')
stats

  0%|          | 0/12 [00:00<?, ?it/s]

CPU times: total: 1min 29s
Wall time: 2min 19s


Start                     2023-09-20 02:15:00
End                       2023-10-25 15:50:00
Duration                     35 days 13:35:00
Exposure Time [%]                   26.317845
Equity Final [$]                 10525.775119
Equity Peak [$]                  14794.250696
Return [%]                         426.288756
Buy & Hold Return [%]               10.221771
Return (Ann.) [%]            854132507.163688
Volatility (Ann.) [%]     1115417130371689...
Sharpe Ratio                              0.0
Sortino Ratio                  4263118.023729
Calmar Ratio                  10499465.140996
Max. Drawdown [%]                  -81.350097
Avg. Drawdown [%]                  -16.645469
Max. Drawdown Duration       24 days 04:15:00
Avg. Drawdown Duration        1 days 12:10:00
# Trades                                  357
Win Rate [%]                        36.134454
Best Trade [%]                       2.076043
Worst Trade [%]                     -0.897445
Avg. Trade [%]                    

In [12]:
stats.keys()

Index(['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'Sharpe Ratio',
       'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]',
       'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration',
       '# Trades', 'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]',
       'Avg. Trade [%]', 'Max. Trade Duration', 'Avg. Trade Duration',
       'Profit Factor', 'Expectancy [%]', 'SQN', '_strategy', '_equity_curve',
       '_trades'],
      dtype='object')

In [13]:
stats._strategy

<Strategy Scalping_Strategy(atr_sl_rate=1.1,TPSL_rate=2.3,size_multiplier_rate=0.4)>

In [14]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stats['_equity_curve'].index, y=stats['_equity_curve']['Equity'],name='Equity')])
#add a initial cash line
fig.add_trace(go.Scatter(x=stats['_equity_curve'].index, y=[init_cash]*len(stats['_equity_curve'].index), name='Initial Cash'))
fig.update_layout(title='Profit |'+symbol, yaxis_title='USDT', xaxis_title='Date')
fig.show()